### Librerias

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import holidays
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

from functools import partial# import ace_tools as tools

### Directorios


In [16]:
DATOS_DIR = '../../data/'
SALIDAS_DIR = '../../salidas/'

# LECTURA DE DATASET DE 16.6 MILLONES PARA TRANSFORMARLO POR PRODUCTO
ESTE SCRIPT SE CORRE POR UNICA VEZ


In [60]:
FILE_NAME_SELL_IN='sell_in_fixed36.parquet'
FILE_NAME_PRODUCTOS='tb_productos.txt'

df = pd.read_parquet(DATOS_DIR + FILE_NAME_SELL_IN)
df_productos = pd.read_csv(DATOS_DIR + FILE_NAME_PRODUCTOS , sep='\t')

### Creo el nuevo dataset agrupado por productos_id y periodos
TOTAL DE PRODUCTO 780 X 36 PERIODOS PARA CADA UNO

In [63]:
# GUARDO EN UN DATASET # LOS REGISTROS QUE TENGAN TN > 0, AGRUPADOS POR product_id, customer_id Y periodo_dt

# Filtrar registros donde tn > 0
df_filtrado = df[df['tn'] > 0]

# Agrupar por product_id, customer_id y periodo_dt, y contar los registros
df_Customer_Compraron = df_filtrado.groupby(['product_id', 'periodo_dt'], as_index=False).size()

# Renombrar la columna de conteo
df_Customer_Compraron.rename(columns={'size': 'cantidad_registros'}, inplace=True)

# Mostrar el resultado
print(df_Customer_Compraron.head(40))

    product_id periodo_dt  cantidad_registros
0        20001 2017-01-01                 186
1        20001 2017-02-01                 185
2        20001 2017-03-01                 188
3        20001 2017-04-01                 104
4        20001 2017-05-01                 238
5        20001 2017-06-01                 220
6        20001 2017-07-01                 151
7        20001 2017-08-01                 236
8        20001 2017-09-01                 163
9        20001 2017-10-01                  96
10       20001 2017-11-01                 175
11       20001 2017-12-01                 192
12       20001 2018-01-01                 152
13       20001 2018-02-01                 157
14       20001 2018-03-01                 204
15       20001 2018-04-01                 154
16       20001 2018-05-01                 188
17       20001 2018-06-01                 188
18       20001 2018-07-01                 167
19       20001 2018-08-01                 193
20       20001 2018-09-01         

In [49]:
# borro columnas que no voy a usar
columnas_a_borrar = ["customer_id","is_synth", "is_synth_36"]
df.drop(columns=columnas_a_borrar, inplace=True, errors='ignore')
# Agrupar por product_id y periodo_dt, y sumar el resto de los campos
df_agrupado = df.groupby(['product_id', 'periodo_dt'], as_index=False).sum()
df_exportar = pd.merge(df_agrupado, df_productos, on='product_id', how='left')

# Exportar el DataFrame a un archivo Parquet

FILE_NAME_SOLO_PRODUCTOS ='sell_in_ByProductos.parquet'
df_exportar.to_parquet(DATOS_DIR + FILE_NAME_SOLO_PRODUCTOS, index=False)

# COMIENZO FE CON DATASET POR PRODUCTOS

In [50]:
# lECTURA DEL ARCHIVO PARQUET
FILE_NAME_SOLO_PRODUCTOS ='sell_in_ByProductos.parquet'
df = pd.read_parquet(DATOS_DIR + FILE_NAME_SOLO_PRODUCTOS)  

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28080 entries, 0 to 28079
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   product_id             28080 non-null  int32         
 1   periodo_dt             28080 non-null  datetime64[ns]
 2   plan_precios_cuidados  28080 non-null  int64         
 3   cust_request_qty       28080 non-null  float64       
 4   cust_request_tn        28080 non-null  float32       
 5   tn                     28080 non-null  float32       
 6   cat1                   28080 non-null  object        
 7   cat2                   28080 non-null  object        
 8   cat3                   28080 non-null  object        
 9   brand                  28080 non-null  object        
 10  sku_size               28080 non-null  int64         
 11  descripcion            28080 non-null  object        
dtypes: datetime64[ns](1), float32(2), float64(1), int32(1), int6

In [ ]:
# agrupo por product_id, customer_id y periodo_dt, y cuento los registros cuando la suma de tn >0
df_agrupado = df.groupby(['product_id', 'customer_id', 'periodo_dt'], as_index=False).sum()

### Agrego periodo de 0 a 35

In [52]:
periodo_mapping = {
    pd.to_datetime("2017-01-01"): 0, pd.to_datetime("2017-02-01"): 1, pd.to_datetime("2017-03-01"): 2,
    pd.to_datetime("2017-04-01"): 3, pd.to_datetime("2017-05-01"): 4, pd.to_datetime("2017-06-01"): 5,
    pd.to_datetime("2017-07-01"): 6, pd.to_datetime("2017-08-01"): 7, pd.to_datetime("2017-09-01"): 8,
    pd.to_datetime("2017-10-01"): 9, pd.to_datetime("2017-11-01"): 10, pd.to_datetime("2017-12-01"): 11,
    pd.to_datetime("2018-01-01"): 12, pd.to_datetime("2018-02-01"): 13, pd.to_datetime("2018-03-01"): 14,
    pd.to_datetime("2018-04-01"): 15, pd.to_datetime("2018-05-01"): 16, pd.to_datetime("2018-06-01"): 17,
    pd.to_datetime("2018-07-01"): 18, pd.to_datetime("2018-08-01"): 19, pd.to_datetime("2018-09-01"): 20,
    pd.to_datetime("2018-10-01"): 21, pd.to_datetime("2018-11-01"): 22, pd.to_datetime("2018-12-01"): 23,
    pd.to_datetime("2019-01-01"): 24, pd.to_datetime("2019-02-01"): 25, pd.to_datetime("2019-03-01"): 26,
    pd.to_datetime("2019-04-01"): 27, pd.to_datetime("2019-05-01"): 28, pd.to_datetime("2019-06-01"): 29,
    pd.to_datetime("2019-07-01"): 30, pd.to_datetime("2019-08-01"): 31, pd.to_datetime("2019-09-01"): 32,
    pd.to_datetime("2019-10-01"): 33, pd.to_datetime("2019-11-01"): 34, pd.to_datetime("2019-12-01"): 35
}

# Agregar la columna periodo_indice al DataFrame usando el mapeo
df['periodo_indice'] = df['periodo_dt'].map(periodo_mapping)

### Agrego nuevas caracteristicas basadas en calendario

In [53]:
df["year"]        = df["periodo_dt"].dt.year.astype("int16")
df["month"]       = df["periodo_dt"].dt.month.astype("int8")
df["quarter"]     = df["periodo_dt"].dt.quarter.astype("int8")
df["month_idx"]   = (df["year"] * 12 + df["month"]).astype("int32")  # continuous index
df["is_fy_start"] = df["month"].eq(1).astype("int8")
df["is_fy_end"]   = df["month"].eq(12).astype("int8")

In [54]:

# -----------------------------------------------------------
# 1. Seasons  (southern hemisphere)
#    Summer = Dec–Feb, Autumn = Mar–May, Winter = Jun–Aug, Spring = Sep–Nov
# -----------------------------------------------------------
season_map = {
    12: "summer", 1: "summer", 2: "summer",
    3:  "autumn", 4: "autumn", 5: "autumn",
    6:  "winter", 7: "winter", 8: "winter",
    9:  "spring", 10: "spring", 11: "spring",
}

df["season"] = (
    df["periodo_dt"].dt.month.map(season_map)
      .astype("category")              # LightGBM friendly
)

In [ ]:
# VER DE AGREGAR CARACTERISTICAS QUE INDIQUEN VACACIONES Y FERIADOS, NAVIDAD, ETC.
# ar_holidays = holidays.AR()
# for ptr in holidays.AR(years = 2018).items():
#     p, int(ptr)

### AGREGO MESES EXTRAÑOS EN EL DATASET

Mes atípico:
- Junio 2019
- Agosto 2019

In [55]:
df["mes_anormal_1"] = (
    df["periodo_dt"].dt.to_period("M") == pd.Period("2019-08", freq="M")
).astype("int8")          # 1 para agosto-2019, 0 para el resto

df["mes_anormal_2"] = (
    df["periodo_dt"].dt.to_period("M") == pd.Period("2019-06", freq="M")
).astype("int8")          # 1 para agosto-2019, 0 para el resto

### Creo clase y lags

In [57]:
df = df.sort_values(["product_id","periodo_dt"])

# tn adelantado 2 periodos (h=2)
df["clase"] = (
    df.groupby(["product_id"])["tn"]
      .shift(-2)          # -2 = lead de 2 meses
      .astype("float32")  # mismo tipo que tn para ahorrar RAM
)

# Generar columnas tn-1, tn-2, ..., tn-11 (12 meses de adelanto)
for i in range(1, 36):
    df[f"tn-{i}"] = (
        df.groupby(["product_id"])["tn"]
          .shift(+i)          # -i = lead de i meses
          .astype("float32")  # mismo tipo que tn para ahorrar RAM
    )


### Rolling: media y desviación en ventana de 3,6 y 12 meses
shift(1) para evitar "ver el futuro"

In [58]:
grp = df.groupby(["product_id"])
# ───────────────────────────────
# Rolling: media y desviación en ventana de 3,6 y 12 meses
# ───────────────────────────────
window_sizes = [3, 6, 12]

for w in window_sizes:
    # Media móvil
    df[f"tn_roll_mean_{w}"] = (
        grp["tn"]
          .rolling(window=w)
          .mean()
          .shift(1)                       # deja fuera el mes actual
          .reset_index(level=[0,1], drop=True)
          .astype("float32")
    )
    # Desviación estándar móvil
    df[f"tn_roll_std_{w}"] = (
        grp["tn"]
          .rolling(window=w)
          .std()
          .shift(1)
          .reset_index(level=[0,1], drop=True)
          .astype("float32")
    )

# ───────────────────────────────
# 3. Opcional: rellenar NaN iniciales con 0 o dejarlos
# ───────────────────────────────
# lag_cols = [f"tn_lag{l}" for l in lag_list]
roll_cols = [f"tn_roll_mean_{w}" for w in window_sizes] + \
            [f"tn_roll_std_{w}"  for w in window_sizes]

df[roll_cols] = df[roll_cols].fillna(0)

# Guardar si quieres
# df.to_parquet("data/sell_in_fixed36_feats.parquet", compression="zstd")


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28080 entries, 0 to 28079
Data columns (total 64 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   product_id             28080 non-null  int32         
 1   periodo_dt             28080 non-null  datetime64[ns]
 2   plan_precios_cuidados  28080 non-null  int64         
 3   cust_request_qty       28080 non-null  float64       
 4   cust_request_tn        28080 non-null  float32       
 5   tn                     28080 non-null  float32       
 6   cat1                   28080 non-null  object        
 7   cat2                   28080 non-null  object        
 8   cat3                   28080 non-null  object        
 9   brand                  28080 non-null  object        
 10  sku_size               28080 non-null  int64         
 11  descripcion            28080 non-null  object        
 12  periodo_indice         28080 non-null  int64         
 13  y

In [11]:
# borro columnas que no voy a usar
columnas_a_borrar = ["periodo_dt", "is_synth", "is_synth_36"]
df.drop(columns=columnas_a_borrar, inplace=True, errors='ignore')

### GUARDO UN PRIMER DATASET EN BASE A ESTE NOTEBOOK

In [13]:
FILE_NAME_SALIDA='sell_in_03-featureaddv1.parquet"'
df.to_parquet(SALIDAS_DIR + FILE_NAME_SALIDA , compression="zstd")